## Librerías

In [171]:
using Pkg; Pkg.add("SciPy"); Pkg.add("Distributions"); Pkg.add("StatsBase")

    Updating registry at `C:\Users\giles\.julia\registries\General.toml`
   Resolving package versions...
  No Changes to `C:\Users\giles\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\giles\.julia\environments\v1.7\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\giles\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\giles\.julia\environments\v1.7\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\giles\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\giles\.julia\environments\v1.7\Manifest.toml`


In [172]:
using SciPy
using Statistics
using StatsBase
using Random
using LinearAlgebra

## Funciones

In [173]:
# Función para generar pares de ángulo polar y azimutal
function generar_angulos()
    theta = 2*pi*rand() # El ángulo polar
    phi = pi*rand() # El ángulo azimutal
    return theta, phi
end

# Transformación de coordenadas esféricas a cartesianas
function transformar_coordenadas(theta::Float64, phi::Float64, r::Any)
    x = r*sin(phi)*cos(theta)
    y = r*sin(phi)*sin(theta)
    z = r*cos(phi)
    return x, y, z
end


transformar_coordenadas (generic function with 2 methods)

In [174]:
# Se define una distribución de Maxwell 3D
function maxwell_3d(x::Any, m::Any, kT::Any)
    down = 2*kT
    valor = 4*pi*((m/(pi*down))^1.5)*(x^2)*exp(-m*(x^2)/down)
    return valor
end

# Dada una distribución, se obtienen puntos aleatorios
function samplear(vector::Array, funcion::Function)
    pesos = map(funcion, vector)
    return sample(vector, Weights(pesos))
end

samplear (generic function with 2 methods)

In [194]:
# La magnitud de la fuerza de Lennard-Jones
function lennard_jones(r::Any, sigma::Any, epsilon=1.0)::Float64
    fraccion = (sigma/r)^6.0
    return 24.0*epsilon*fraccion*(2.0*fraccion-1.0)/r
end

# La magnitud de la fuerza basada en el potencial WCA
function wca(r, sigma, epsilon=1.0)::Float64
    if r<=(2.0^(1/6))*sigma
        return lennard_jones(r, sigma, epsilon)
    end
    return 0.0
end

wca (generic function with 2 methods)

In [176]:
# Genera las posiciones de las partículas sin solaparse
function generar_posiciones(n_particulas::Int64, tamanio::Any,
    distribucion_r::Function, distribucion_m::Function, distribucion_sigma::Function)
    masas = []
    sigmas = []
    posiciones = []

    i, intentos = 0, 0
    while i<n_particulas
        intentos += 1

        sigma_prueba = distribucion_sigma(0)

        theta, phi = generar_angulos()
        magnitud = tamanio*distribucion_r(0)
        r_prueba = mod.(transformar_coordenadas(theta, phi, magnitud), tamanio)

        coincide = true
        for j in Array(1:length(posiciones))
            distancia = norm(r_prueba.-posiciones[j])
            if distancia < sigma_prueba+sigmas[j]
                coincide = false
                break
            end
        end

        if coincide
            i += 1
            push!(posiciones, r_prueba)
            append!(sigmas, sigma_prueba)
            append!(masas, distribucion_m(0))
        end

        if intentos>n_particulas^3
            i = 0
            intentos = 0
            masas = []
            posiciones = []
            sigmas = []
        end
    end
    
    return sigmas, posiciones, masas
end

# Genera las velocidades de las partículas
function generar_velocidades(n_particulas::Int64, m::Any, kT::Any, distribucion_v::Function)
    velocidades = []
    for i in Array(1:n_particulas)
        theta, phi = generar_angulos()
        magnitud = distribucion_v(m, kT)
        push!(velocidades, transformar_coordenadas(theta, phi, magnitud))
    end
    return velocidades
end

generar_velocidades (generic function with 1 method)

In [202]:
# Generar la simulación
function iniciar_simulacion(n_particulas=1000, tamanio=1, T=1, k=1,
                            distribucion_r = x::Int64 -> begin rand() end,
                            distribucion_v = (m::Any, kT::Any) -> begin
                                                                        samplear(Array(1:10e3)./10e3, 
                                                                        y::Any -> begin maxwell_3d(y, m, kT) end)
                                                                    end,
                            distribucion_m = x::Int64 -> begin 1.0 end,
                            distribucion_sigma = x::Int64 -> begin 0.0001 end)

    # Primero se valida si es posible meter tantas esferas en una caja
    mean_sigma = mean([distribucion_sigma(0) for i in Array(1:100)]) # El diámetro promedio
    densidad_empaquetamiento_max = pi/(3*sqrt(2))

    volumen_esferas = n_particulas*(pi/3)*(mean_sigma^3/2)
    volumen_maximo =  densidad_empaquetamiento_max*(tamanio^3)

    if volumen_esferas<volumen_maximo

        sigmas, posiciones, masas = generar_posiciones(n_particulas, tamanio,
                                                       distribucion_r, distribucion_m,
                                                       distribucion_sigma)

        mean_m = mean(masas)

        velocidades = generar_velocidades(n_particulas, mean_m, k*T, distribucion_v)

        simulacion = [[posiciones[i][1], posiciones[i][2], posiciones[i][3],
                       velocidades[i][1], velocidades[i][2], velocidades[i][3],
                       0.0, 0.0, 0.0, masas[i], sigmas[i]] for i in Array(1:n_particulas)]
                       
        return simulacion
    else
        println("No es posible meter tantas esferas en una caja de ese tamaño")
        return []
    end
end


# Avanza la simulación dada una fuerza interna
function avanzar_simulacion(simulacion::Vector{Vector{Float64}}, tamanio=1, dt::Float64=0.00001,
                            fuerza_interna = (r::Any, sigma::Any) -> begin float(lennard_jones(r, sigma, 1.0)) end)

    n_particulas = length(simulacion)

    if n_particulas>0
        indices = Array(1:n_particulas)
        aceleraciones = [[0.0, 0.0, 0.0] for i in indices]

        for i in Array(1:(n_particulas-1))
            for j in Array((i+1):n_particulas)
                r_i = simulacion[i][1:3]
                r_j = simulacion[j][1:3]
                
                r_hat = r_j-r_i
                distancia = norm(r_hat)
                r_hat = r_hat/distancia

                sigma = (simulacion[i][11]+simulacion[j][11])/2
                fuerza_resultante = fuerza_interna(distancia, sigma).*r_hat

                aceleraciones[i] = aceleraciones[i] + (fuerza_resultante./simulacion[i][10])
                aceleraciones[j] = aceleraciones[j] - (fuerza_resultante./simulacion[j][10])
            end
        end

        simulacion_avanzada = simulacion[:]

        for i in indices
            r_i = simulacion[i][1:3]
            v_i = simulacion[i][4:6]
            a_i0 = simulacion[i][7:9]
            a_i1 = aceleraciones[i][:]
      
            simulacion_avanzada[i][1:3] = mod.(r_i + (dt.*v_i) + ((dt^2/2).*a_i0), tamanio)
            simulacion_avanzada[i][4:6] = v_i + ((dt/2).*(a_i0+a_i1))
            simulacion_avanzada[i][7:9] = a_i1
        end

        return simulacion_avanzada
    else
        println("No hay partículas en la simulación")
        return []
    end
end

avanzar_simulacion (generic function with 15 methods)

# Pruebas

In [178]:
sim = iniciar_simulacion()

1000-element Vector{Vector{Float64}}:
 [0.7832446378203192, 0.7146686070386594, 0.7435660414622356, -0.09644684196292642, -0.24847121102457914, -0.03850394742565163, 0.0, 0.0, 0.0, 1.0, 0.0001]
 [0.04196688354603019, 0.2977025844047574, 0.0866491355564382, -0.08581300403908533, 0.05131062473093512, -0.8237544464993856, 0.0, 0.0, 0.0, 1.0, 0.0001]
 [0.8996404725475292, 0.1751006058932, 0.12375400058583202, 0.16316216156165206, -0.28805308887816194, -0.6663995325795773, 0.0, 0.0, 0.0, 1.0, 0.0001]
 [0.04041651454675021, 0.9220375093892412, 0.09477501429172984, 0.014053463769869792, 0.039908796305675676, 0.6840927920490765, 0.0, 0.0, 0.0, 1.0, 0.0001]
 [0.020405993612331114, 0.1682469526888134, 0.044126447272445676, 0.04398856003329881, 0.08593326341903086, -0.5481642462112539, 0.0, 0.0, 0.0, 1.0, 0.0001]
 [0.8812516649405294, 0.007226923557602442, 0.1675418145479559, -0.42128537082839823, -0.35103862021220766, -0.06660543105099234, 0.0, 0.0, 0.0, 1.0, 0.0001]
 [0.2251784303832478, 0.8708

In [203]:
avanzar_simulacion(sim)

1000-element Vector{Vector{Float64}}:
 [0.7832427088834798, 0.7146636376144389, 0.743565271383287, -0.09644684196292642, -0.24847121102457914, -0.03850394742565163, 5.1987223853624e-15, -4.2515583102158136e-14, -1.2128630247304858e-14, 1.0, 0.0001]
 [0.041965167285949415, 0.29770361061725203, 0.08663266046750821, -0.08581300403908533, 0.05131062473093512, -0.8237544464993856, 5.971018544443548e-15, -4.100134538761995e-15, 4.7576182700186216e-15, 1.0, 0.0001]
 [0.8996437357907604, 0.17509484483142243, 0.12374067259518043, 0.16316216156165206, -0.28805308887816194, -0.6663995325795773, 2.161859541842376e-14, -3.632554661126566e-14, 8.234737087963193e-16, 1.0, 0.0001]
 [0.040416795616025604, 0.9220383075651672, 0.09478869614757081, 0.014053463769869792, 0.039908796305675676, 0.6840927920490765, -5.2790237054302704e-15, -2.198744199194307e-14, 4.9194367951890326e-14, 1.0, 0.0001]
 [0.02040687338353178, 0.16824867135408178, 0.04411548398752145, 0.04398856003329881, 0.08593326341903086, -0.5